In [6]:
import itertools

def convert(List):
    P = List[:]
    for i in range(len(P)):
        l = P[i]
        newL = []
        for c in l:
            newL.append(int(c))
    #         if c == '0':
    #             newL.append('N')
    #         if c == '1':
    #             newL.append('R')
    #         if c == '2':
    #             newL.append('HR')
        P[i] = newL
    return P

P = ["".join(item) for item in itertools.product("012", repeat=5)]
E =  ["".join(item) for item in itertools.product("012", repeat=5)]

P = convert(P)
E = convert(E)

L = []
for item1 in P:
    for item2 in E:
        pair = []
        pair.append(item1)
        pair.append(item2)
        L.append(pair)

In [4]:
import math
#Precision at K
def precision(res, k):
    counter = 0
    for i in range(k):
        if res[i] != 0:
            counter += 1
    return 1.0*counter / k

def DCG(res, k):
    sum = 0
    for r in range(1,k+1):
        sum += (2**res[r-1] - 1)/math.log(1+r,2)
    return sum

def RBP(res, theta = 0.8):
    sum = 0
    for k in range(1, len(res)):
        sum += res[k-1] * theta**(k-1)*(1 - theta)
    return sum

print L[45]
print precision(L[45][0],k = 3),',', precision(L[45][1],k = 3)
print DCG(L[45][0],k = 3),',',DCG(L[45][1],k = 3)
print DCG([5,5,0,0,1,0,0,0,0,0], 10)
print RBP([0,1,2,1,0])#L[45][1])

[[0, 0, 0, 0, 0], [0, 1, 2, 0, 0]]
0.0 , 0.666666666667
0.0 , 2.13092975357
50.9456751679
0.5184


In [7]:
def measure(pairs, measure, k=5):
    result = []
    for pair in pairs:
        if measure == "precision":
            P = precision(pair[0], k)
            E = precision(pair[1], k)
        
        if measure == "DCG":
            P = DCG(pair[0], k)
            E = DCG(pair[1], k)
            
        if measure == "RBP":
            P = RBP(pair[0])
            E = RBP(pair[1])
         
        diff = E - P
        
        if diff > 0:
            result.append([pair, diff])
            
    return result

res = measure(L, "DCG")
print L[5432]
print res[5432]

print DCG([2, 0, 2, 2, 2], 5) - DCG([0, 0, 2, 2, 1], 5)

[[0, 0, 2, 1, 1], [1, 0, 0, 1, 2]]
[[[0, 0, 2, 2, 1], [2, 0, 2, 2, 2]], 3.7737056144690833]
3.77370561447


In [32]:
import random
import numpy

def teamDraftInterleaving(pair, clicks):
    l1 = zip(pair[0],[1]*len(pair[0]))
    l2 = zip( pair[1],[2]*len(pair[1]))
    combined = zip(l1, l2)
    shuffled = []
    for line in combined:
        lst = list(line)
        random.shuffle(lst)
        shuffled.append(lst)
    flattened = [item for sublist in shuffled for item in sublist]
    c1 = 0
    c2 = 0
    for index in clicks:
        if flattened[index][1] == 1:
            c1+=1
        if flattened[index][1] == 2:
            c2+=1
          
    print c1, c2
    return numpy.sign(c1-c2)
                

teamDraftInterleaving(L[1333], [3, 3])

0 2


-1